In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import re
from tqdm import tqdm
from deepxml.data_utils import build_vocab, convert_to_binary

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

from sklearn.preprocessing import MultiLabelBinarizer, normalize

from sklearn.model_selection import train_test_split

ImportError: cannot import name 'convert_to_binary' from 'deepxml.data_utils' (/home/daniil/programming/PapersTagsPrediction/src/deepxml/data_utils.py)

In [3]:
df = pd.read_parquet("../data/habr_posts_dataset.parquet")

In [4]:
df

,post_id,author,title,tags,text
0,807711,Kaspersky_Lab,Security Week 2416: уязвимость в серверных мат...,"[Блог компании «Лаборатория Касперского», Инфо...",На прошлой неделе исследователи компании Binar...
1,807709,markshevchenko,Вычислительные выражения: Подробнее про типы-о...,"[.NET, Функциональное программирование, F#]",В предыдущем посте мы познакомились с концепци...
2,807707,ru_vds,Угадай местоположение льдины с арктическим ЦОД...,"[Блог компании RUVDS.com, Хостинг, Системное а...","Как вы наверняка знаете, 12 апреля RUVDS успеш..."
3,807705,shaddyk,Запустили проект с НСИС по повышению качества ...,"[Блог компании HFLabs, Открытые данные, IT-ком...",НСИС — оператор единой автоматизированной инфо...
4,807703,VokaMut,Тестируем AI на создании прикладного приложения,"[Веб-разработка, Искусственный интеллект, Natu...","Всем привет, я Григорий Тумаков, CTO в Моризо ..."
...,...,...,...,...,...
4371,797723,Squirrelfm,Анатомия эффективного собеседования. Что делат...,"[Блог компании Raft, Управление персоналом, Ка...","Я провел много собеседований за свою карьеру, ..."
4372,797721,maybe_elf,None,"[Законодательство в IT, Искусственный интеллек...",В OpenAI рассказали о мотивах Илона Маска при ...
4373,797719,ar4w,SD-Access без DNAC и ISE,"[Информационная безопасность, IT-инфраструктур...",В 2019 мы закупили комплект оборудования и лиц...
4374,797715,maybe_elf,Meta* удалит все учётные записи Oculus в конце...,"[Управление сообществом, Разработка под AR и V...",Meta* в электронной рассылке сообщила пользова...


In [5]:
def tokenize(sentence: str, sep='/SEP/'):
    # We added a /SEP/ symbol between titles and descriptions such as Amazon datasets.
    return [token.lower() if token != sep else token for token in word_tokenize(sentence)
            if len(re.sub(r'[^\w]', '', token)) > 0]

In [6]:
tokenized = []
for post in tqdm(df["text"]):
    tokenized.append(tokenize(post))

  0%|          | 0/4376 [00:00<?, ?it/s]

100%|██████████| 4376/4376 [00:23<00:00, 183.48it/s]


In [7]:
vocab, emb_init = build_vocab([" ".join(t) for t in tokenized],
                              vocab_size=500_000)

In [10]:
vocab = {word: i for i, word in enumerate(vocab)}

In [200]:
texts, labels = convert_to_binary([" ".join(t) for t in tokenized], 
                                  df["tags"].to_list(), 1000, vocab)



























































































Converting token to id: 100%|██████████| 4376/4376 [00:03<00:00, 1360.05it/s]



Converting labels: 100%|██████████| 4376/4376 [00:00<00:00, 430588.71it/s]


In [201]:
labels

[['Блог компании «Лаборатория Касперского»', 'Информационная безопасность'],
 ['.NET', 'Функциональное программирование', 'F#'],
 ['Блог компании RUVDS.com',
  'Хостинг',
  'Системное администрирование',
  'Сетевые технологии',
  'IT-компании'],
 ['Блог компании HFLabs', 'Открытые данные', 'IT-компании'],
 ['Веб-разработка', 'Искусственный интеллект', 'Natural Language Processing'],
 ['Управление персоналом', 'Финансы в IT', 'Транспорт'],
 ['Управление продажами', 'Транспорт', 'Будущее здесь', 'Урбанизм'],
 ['Ненормальное программирование',
  'Open source',
  'Визуальное программирование'],
 ['Работа с видео', 'Управление медиа', 'Видеотехника'],
 ['Управление персоналом',
  'Карьера в IT-индустрии',
  'Статистика в IT',
  'Финансы в IT'],
 ['Интернет-маркетинг', 'Контент и копирайтинг'],
 ['Информационная безопасность',
  'IT-инфраструктура',
  'Сетевые технологии',
  'Софт',
  'Сетевое оборудование'],
 ['Блог компании VK',
  'Мессенджеры',
  'Управление разработкой',
  'Управление пе

In [202]:
mlb = MultiLabelBinarizer(sparse_output=True)

In [203]:
mlb.fit(labels)

MultiLabelBinarizer(sparse_output=True)

In [204]:
labels = mlb.transform(labels)

In [380]:
from torch.utils.data import DataLoader
import torch

from deepxml.dataset import MultiLabelDataset
from deepxml.models import Model
from deepxml.attentionxml import AttentionXML, CorNetAttentionXML
from deepxml.meshprobenet import MeSHProbeNet, CorNetMeSHProbeNet
from deepxml.bertxml import BertXML, CorNetBertXML
from deepxml.xmlcnn import XMLCNN, CorNetXMLCNN

from pathlib import Path

In [381]:
train_texts, val_text, train_labels, val_labels = train_test_split(texts, labels, test_size=0.1)

In [382]:
train_loader = DataLoader(MultiLabelDataset(train_texts, train_labels),
                          8, shuffle=True)

In [383]:
val_loader = DataLoader(MultiLabelDataset(val_text, val_labels),
                          8, shuffle=False)

In [384]:
model = Model(network=CorNetAttentionXML, model_path=str(Path.cwd().parent / "data/models"), **{"hidden_size": 256,
  "layers_num": 1,
  "linear_size": [256],
  "dropout": 0.5,
  "emb_trainable": False},
               labels_num=labels.shape[1], emb_size=300, vocab_size=len(vocab))

In [385]:
model.train(train_loader, val_loader, nb_epoch=10)

saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no attribute 'module'
saving failed
'CorNetAttentionXML' object has no

In [386]:
val_res = model.predict(val_loader)

Predict:   0%|          | 0/55 [00:00<?, ?it/s]

In [387]:
val_res[0].round(2)

array([[0.28, 0.15, 0.11, ..., 0.  , 0.  , 0.  ],
       [0.29, 0.22, 0.09, ..., 0.01, 0.01, 0.01],
       [0.54, 0.2 , 0.13, ..., 0.  , 0.  , 0.  ],
       ...,
       [0.48, 0.4 , 0.21, ..., 0.  , 0.  , 0.  ],
       [0.47, 0.34, 0.32, ..., 0.  , 0.  , 0.  ],
       [0.96, 0.74, 0.51, ..., 0.01, 0.01, 0.01]], dtype=float32)

In [395]:
val_res[1]

array([[141, 389,  93, ..., 113,  98, 429],
       [141, 501, 460, ..., 338, 531, 496],
       [477, 145, 427, ...,  71, 307, 425],
       ...,
       [398, 546, 483, ..., 423, 565, 387],
       [424, 414, 525, ...,   2, 150, 181],
       [305, 571, 572, ...,  48, 576, 153]])

In [389]:
import numpy as np

In [396]:
np.where(val_labels.todense()[2]!=0)

(array([0, 0, 0]), array([426, 477, 576]))

In [397]:
val_res[1][2]

array([477, 145, 427, 466, 576, 149, 586, 588, 582, 400, 460, 492, 240,
       587, 574, 269, 436, 424, 590, 483, 358, 438, 454, 141, 150, 584,
       570, 457, 422, 501, 505, 569, 568, 567, 344, 148, 453, 415, 552,
       439, 476, 437, 531, 473,  31, 565, 589, 226, 317, 270, 461, 102,
       571, 563,  69, 479, 577, 320, 512, 450, 521, 572, 168, 435, 409,
       440, 416, 493, 509, 472,  88, 462, 413, 401, 147, 545,  16, 434,
       181, 496, 441, 430, 414, 481, 499, 426, 142, 328,  93, 179, 508,
       429, 575, 547, 551, 154, 557,  71, 307, 425])

In [399]:
np.where(val_res[1][0]==576)

(array([53]),)

In [221]:
from deepxml.evaluation import get_p_1, get_p_5, get_p_10

In [400]:
val_res[1][:, :10]

array([[141, 389,  93, ..., 501, 460, 524],
       [141, 501, 460, ..., 455,  20,  81],
       [477, 145, 427, ..., 588, 582, 400],
       ...,
       [398, 546, 483, ..., 523, 516, 559],
       [424, 414, 525, ...,  69, 543, 586],
       [305, 571, 572, ..., 589, 501,  73]])

In [403]:
a = np.zeros(shape=val_labels.shape, dtype=int)
a[[[i] for i in range(len(a))], val_res[1][:, :1]] = 1

In [404]:
np.multiply(a, val_labels.todense()).sum()/(1*val_labels.shape[0])

0.4885844748858447